In [99]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
df = train.append(test)
df.set_index('id',inplace=True)
df.head(2)

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331


# EDA
---

There some variables correlated but any of them above 0.9 for other we can deal with it using l1 or l2 penalty

In [3]:
corr_analysis = pd.melt(df.corr(),var_name='variable2',value_name='corr',ignore_index=False).reset_index().rename(columns={'index':'variable1'})
corr_analysis.query('variable1 != variable2',inplace=True)
corr_analysis['corrabs'] = corr_analysis['corr'].abs()
corr_analysis['varTuple'] = corr_analysis.apply(lambda x: str(sorted([x['variable1'],x['variable2']])),axis=1)
corr_analysis.drop_duplicates(["varTuple"],inplace=True)
corr_analysis.sort_values("corrabs").tail(7)

,variable1,variable2,corr,corrabs,varTuple
129,cont10,cont9,0.725733,0.725733,"['cont10', 'cont9']"
85,cont11,cont6,0.737542,0.737542,"['cont11', 'cont6']"
86,cont12,cont6,0.756768,0.756768,"['cont12', 'cont6']"
84,cont10,cont6,0.771422,0.771422,"['cont10', 'cont6']"
87,cont13,cont6,0.784373,0.784373,"['cont13', 'cont6']"
8,cont9,cont1,0.793965,0.793965,"['cont1', 'cont9']"
161,cont12,cont11,0.825740,0.825740,"['cont11', 'cont12']"


# Model Development

---



## Feature Engineering 1

### Polynomial Features

### Multiplied Features

## Cross Validation

We gonna apply Kfold as our CV strategy 

In [4]:
train = df.dropna()
train

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.216974,0.735265,0.648648,0.255387,0.616353,0.345197,0.295718,0.304357,0.314351,0.860504,0.315397,0.247682,0.486542,0.288750,7.385215
499996,0.545799,0.165139,0.220966,0.190053,0.359362,0.386336,0.365767,0.344217,0.466446,0.454581,0.360251,0.360755,0.292535,0.619984,7.242617
499997,0.284401,0.841542,0.957585,0.340383,0.396279,0.330376,0.525687,0.260039,0.378174,0.526925,0.491735,0.516629,0.173521,0.714552,9.592487


In [104]:
from sklearn.model_selection import KFold,GridSearchCV,train_test_split
cv = KFold(3,shuffle=True,random_state=42)
x = train.drop('target',axis=1)
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

## Hyperparameter Tunning

In [105]:
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.metrics import make_scorer,mean_squared_error
mse = make_scorer(mean_squared_error,greater_is_better=False)

In [106]:
params_dict = {
    "ElasticNet":{
        'alpha':[0,0.3,0.5,0.7,1],
        'l1_ratio':[0,0.3,0.5,0.7,1],
    },
    "DecisionTreeRegressor":{
        'max_depth':[20,40,70,100,130,None],
        'min_samples_split':[2,20,100,300,1000]
    },
    "MLPRegressor":{
        'activation':['relu'],
        "hidden_layer_sizes":[(100,),(40,)],
        "alpha":[0.01,0.001]
        "early_stopping":[True]
    },
    "HistGradientBoostingRegressor":{
        "l2_regularization":10. ** (-np.arange(1,3)),
        "max_iter":[10,30,100,300],
        "learning_rate":10. ** (-np.arange(1,3))
    },
    "GaussianProcessRegressor":{
        "alpha":10. ** (-np.arange(8,10)),
    },
    "KNeighborsRegressor":{
        "n_neighbors":[3,5,7,11],
        "weights":['uniform','distance']
    }
}

model_dict = {
    "ElasticNet":GridSearchCV(ElasticNet(random_state=42),param_grid=""),
    "DecisionTreeRegressor":GridSearchCV(DecisionTreeRegressor(random_state=42),param_grid=""),
    "MLPRegressor":GridSearchCV(MLPRegressor(random_state=42),param_grid=""),
    "HistGradientBoostingRegressor":GridSearchCV(HistGradientBoostingRegressor(random_state=42),param_grid=""),
    #"GaussianProcessRegressor":GridSearchCV(GaussianProcessRegressor(random_state=42),param_grid=""),
    "KNeighborsRegressor":GridSearchCV(KNeighborsRegressor(n_jobs=-1),param_grid="")
}



In [107]:
for key,item in model_dict.items():
    model_dict[key].param_grid = params_dict[key]
    model_dict[key].cv = cv
    model_dict[key].verbose = 5
    model_dict[key].return_train_score=True
    model_dict[key].scoring = mse
    #model_dict[key].n_jobs = -1

## Models

We gonna get the results from the models and use them as variables to the next level of the model and drop the others

In [108]:
evaluation = pd.DataFrame()
for key,item in model_dict.items():
    print("-----------")
    print(key)
    model_dict[key].fit(X_train,y_train)
    results = pd.DataFrame(model_dict[key].cv_results_)
    results['model'] = key
    evaluation = evaluation.append(results)

-----------
ElasticNet
Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] alpha=0, l1_ratio=0 .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  alpha=0, l1_ratio=0, score=(train=-0.528, test=-0.529), total=   2.9s
[CV] alpha=0, l1_ratio=0 .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s


[CV]  alpha=0, l1_ratio=0, score=(train=-0.529, test=-0.527), total=   2.9s
[CV] alpha=0, l1_ratio=0 .............................................


...

[CV]  l2_regularization=0.1, learning_rate=0.1, max_iter=30, score=(train=-0.501, test=-0.505), total=   0.8s
[CV] l2_regularization=0.1, learning_rate=0.1, max_iter=30 ...........
[CV]  l2_regularization=0.1, learning_rate=0.1, max_iter=30, score=(train=-0.500, test=-0.506), total=   0.7s
[CV] l2_regularization=0.1, learning_rate=0.1, max_iter=100 ..........
[CV]  l2_regularization=0.1, learning_rate=0.1, max_iter=100, score=(train=-0.479, test=-0.496), total=   1.6s
[CV] l2_regularization=0.1, learning_rate=0.1, max_iter=100 ..........
[CV]  l2_regularization=0.1, learning_rate=0.1, max_iter=100, score=(train=-0.480, test=-0.494), total=   1.6s
[CV] l2_regularization=0.1, learning_rate=0.1, max_iter=100 ..........
[CV]  l2_regularization=0.1, learning_rate=0.1, max_iter=100, score=(train=-0.479, test=-0.496), total=   1.6s
[CV] l2_regularization=0.1, learning_rate=0.1, max_iter=300 ..........
[CV]  l2_regularization=0.1, learning_rate=0.1, max_iter=300, score=(train=-0.449, test=-0.4

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  1.8min finished


-----------
GaussianProcessRegressor
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] alpha=1e-08 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


MemoryError: 

In [110]:
key = "KNeighborsRegressor"
model_dict[key].fit(X_train,y_train)
results = pd.DataFrame(model_dict[key].cv_results_)
results['model'] = key
evaluation = evaluation.append(results)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] n_neighbors=3, weights=uniform ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [111]:
best_models = evaluation.sort_values('rank_test_score',ascending=True)[['model','mean_train_score','mean_test_score']].drop_duplicates('model')
best_models['mean_train_score'] = np.sqrt(np.abs(best_models['mean_train_score']))
best_models['mean_test_score'] = np.sqrt(np.abs(best_models['mean_test_score']))
best_models

,model,mean_train_score,mean_test_score
0,ElasticNet,0.726862,0.726915
14,DecisionTreeRegressor,0.705269,0.721025
14,MLPRegressor,0.713023,0.715238
3,HistGradientBoostingRegressor,0.670531,0.702122


In [112]:
for model_name in best_models['model']:
    model_dict[model_name].best_estimator_.fit(x,y)
best_models['validation_score'] = best_models['model'].apply(lambda model_name: np.sqrt(mean_squared_error(model_dict[model_name].best_estimator_.predict(X_test),y_test)))
best_models

,model,mean_train_score,mean_test_score,validation_score
0,ElasticNet,0.726862,0.726915,0.723482
14,DecisionTreeRegressor,0.705269,0.721025,0.700234
14,MLPRegressor,0.713023,0.715238,0.709411
3,HistGradientBoostingRegressor,0.670531,0.702122,0.680289


## Feature Engineering 2

### Polynomial Features

### Multiplied Features

## Ensembles

## Model Evaluation

## Submissions output

In [114]:
key = "HistGradientBoostingRegressor"
test = df[df['target'].isna()].drop('target',axis=1)
model = model_dict[key].best_estimator_
submissions = pd.DataFrame(model.predict(test),columns=['target'],index=test.index)
submissions.reset_index(inplace=True)
submissions.to_csv(f'submissions/{key}.csv',index=False)

In [48]:
submissions

,id,target
0,0,8.193006
1,2,7.863255
2,6,7.890219
3,7,8.082923
4,10,8.109815
...,...,...
199995,499984,8.076341
199996,499985,7.948479
199997,499987,8.192711
199998,499988,7.969531
